In [ ]:
from skimage import data
from skimage import transform
from math import sqrt
from skimage.feature import blob_dog, blob_log, blob_doh
from pydng.core import RPICAM2DNG
from io import BytesIO
import os
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
def _plot(img):
    plt.figure(figsize=figsize)
    plt.imshow(img)

In [ ]:
image_file = 'testa_1202.jpg'



with open(image_file, 'rb') as fh:
    buf = BytesIO(fh.read())
rpicam = RPICAM2DNG()
raw = rpicam.__extractRAW__(buf)
raw = np.array(raw, dtype=np.float)
print(type(raw))
print(rpicam.__exif__.keys())
print(rpicam.maker_note.keys())

print(raw.shape, raw.size, raw.dtype)

#0.2125 R + 0.7154 G + 0.0721 B

carray = 0.2125*raw[::2,::2] + 0.7154 * (raw[1::2,::2] + raw[::2,1::2]) / 0.5 + 0.0721 * raw[1::2,1::2]
r,c = carray.shape
print("Max is", np.max(carray), "min", np.min(carray))

print(carray[r//2-5:r//2+5,c//2-5:c//2+5])

carray = carray[200:-200,400:-500]

carray /= np.max(carray)

fig = plt.figure(figsize = (20,20))
ax = fig.add_subplot(111)
ax.imshow(carray, cmap='gray', interpolation='none',vmax=0.5)

# ax1 = fig.add_subplot(221)
# ax1.imshow(raw[::2,::2], cmap='gray', interpolation='none', vmin=100, vmax=500)
# ax2 = fig.add_subplot(222)
# ax2.imshow(raw[1::2,::2], cmap='gray', interpolation='none', vmin=100, vmax=500)
# ax3 = fig.add_subplot(223)
# ax3.imshow(raw[::2,1::2], cmap='gray', interpolation='none', vmin=100, vmax=500)
# ax4 = fig.add_subplot(224)
# ax4.imshow(raw[1::2,1::2], cmap='gray', interpolation='none', vmin=100, vmax=500)

plt.show()

In [ ]:
# image = data.hubble_deep_field()[0:500, 0:500]
# image_gray = rgb2gray(image)

image_gray = carray

blobs_log = blob_log(image_gray, max_sigma=30, num_sigma=5, threshold=.03)


maskshape = image_gray.shape
immask = np.zeros(maskshape)
blobmask = np.ones(maskshape)

bsize = 50

for rows in range(maskshape[0]//bsize):
    for cols in range(maskshape[1]//bsize):
        sec = image_gray[bsize*rows:bsize*rows+bsize,bsize*cols:bsize*cols+bsize]
        imave = np.average(sec)
        imstd = np.std(sec)
        mask = np.where(sec > imave+3*imstd, 0, 1)
        immask[bsize*rows:bsize*rows+bsize,bsize*cols:bsize*cols+bsize] = mask

print("Mean", imave, "std", imstd)

# Compute radii in the 3rd column.
blobs_log[:, 2] = blobs_log[:, 2] * sqrt(2)


fig, axes = plt.subplots(2, 1, figsize=(18, 18), sharex=True, sharey=True)
ax = axes.ravel()
ax[0].set_title('Laplacian of Gaussian')
# ax[0].imshow(image_gray, cmap='gray', interpolation='none')
for blob in blobs_log:
    y, x, r = blob
    y = int(y)
    x = int(x)
    if y < 2:
        y = 2
    if x < 2:
        x = 2
    if y > maskshape[0] - 3:
        y = maskshape[0] - 3
    if x > maskshape[1] - 3:
        x = maskshape[1] - 3
    
    blobmask[y-2:y+2,x-2:x+2] = 0*blobmask[y-2:y+2,x-2:x+2]
    c = plt.Circle((x, y), r, color='yellow', linewidth=2, fill=False)
#     ax[0].add_patch(c)
ax[0].imshow(blobmask, cmap='gray', interpolation='none')
ax[0].set_axis_off()


ax[1].imshow(immask, cmap='gray', interpolation='none')
ax[1].set_title("Mask using variance")
ax[1].set_axis_off()


plt.tight_layout()
plt.show()

In [ ]:
x = np.linspace(0.0, 1.0, num=50, endpoint=True)*np.ones((50,50))
x = x.T*x
# print(x)

def linearLine(y, weights):
    '''
    Returns linear regression on the data with weights
    assuming the data is in order
    '''
    N = len(y)
    x = np.arange(N)    
    a = np.vstack([x, np.ones(N)]).T
    #(X.TWX)b = X.TWy
    W = np.diag(weights)
    a_weight = np.dot(a.T, W)
    coef = np.dot(np.linalg.inv(np.dot(a_weight, a)), np.dot(a_weight, y))
    return coef[0]*x+coef[1]

lightnoise = np.zeros(image_gray.shape)

for rows in range(maskshape[0]//bsize):
    for cols in range(maskshape[1]):
        y = image_gray[bsize*rows:bsize*rows+bsize,cols]
        w = blobmask[bsize*rows:bsize*rows+bsize,cols]
        lightnoise[bsize*rows:bsize*rows+bsize,cols] += linearLine(y,w)

print(np.max(lightnoise), np.max(image_gray))
fig = plt.figure(figsize = (20,20))
ax = fig.add_subplot(111)
ax.imshow(image_gray - lightnoise, cmap='gray', interpolation='none',vmax=0.05)
plt.show()

In [ ]:

def warpPerspective(img, M, dsize):
    mtr = to_mtx(img)
    R,C = dsize
    dst = np.zeros((R,C,mtr.shape[2]))
    for i in range(mtr.shape[0]):
        for j in range(mtr.shape[1]):
            res = np.dot(M, [i,j,1])
            i2,j2,_ = (res / res[2] + 0.5).astype(int)
            if i2 >= 0 and i2 < R:
                if j2 >= 0 and j2 < C:
                    dst[i2,j2] = mtr[i,j]
    
    return to_img(dst)
    